# Efficient Requirement Analysis & Specification with LLM Agents

## 🛠️ Config

In this section, we configure the environment, suppress warnings, and load essential packages. The dotenv library allows us to securely load sensitive environment variables from a .env file, keeping credentials and other configuration details safe.

In [ ]:
# Importing necessary libraries

import warnings  # To control warning outputs for a cleaner notebook
warnings.filterwarnings('ignore')  # Suppress all warnings 

from dotenv import load_dotenv  # To load environment variables from a .env file

import os  # To interact with the operating system
import json  # For handling JSON data
import yaml  # For handling YAML data which we will utilize for agent and task definition

## Loading environment variables from the .env file
load_dotenv()  # This will look for the .env file and load the variables

## Alternatively, we can set the environment variables directly using os.environ
# Example:
# os.environ['OPENAI_API_KEY'] = 'xxxxxx'

# Defining the default model name for OpenAI
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'  # Setting the model to be used in subsequent operations

## Single Agent, Single Task Use Case

In this setup, we follow a 1 agent, 1 task framework, where each agent is assigned a unique, specific task. This approach enhances focus and efficiency, ensuring that each agent performs optimally by concentrating on their specialized assignment.

Here's a breakdown:
- 🤖 Agent Definition: Each agent is carefully designed with a clear role, goal, and backstory that align with their unique strengths and expertise. This alignment allows the agent to approach tasks with a deep understanding of their responsibilities and objectives. For more information, refer to the full list of [agent's attributes](https://docs.crewai.com/concepts/agents#agent-attributes).
- ✅ Task Specification: Tasks are explicitly tailored to each agent’s capabilities, with well-defined description and expected_output attributes. This clarity allows agents to work independently, producing high-quality deliverables without requiring additional guidance. Refer to the complete list of [task's attributes](https://docs.crewai.com/concepts/tasks#task-attributes) for detailed task configuration.

🔋 This setup creates a streamlined workflow by establishing a one-to-one mapping between agents and tasks, minimizing overlap and redundancy. Each agent focuses solely on their assigned task, resulting in a structured, organized, and effective process that promotes high-quality outcomes.

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

### 🧰 Tools Definition

Our toolkit includes a blend of CrewAI, Langchain, LlamaIndex, and custom tools designed for specialized search and retrieval tasks.

#### 🔍 Search Tools

- 🌐 **SerperDevTool**: Searches the internet, returning the most relevant Google results using the Serper API.

- 🕸️ **WebsiteSearchTool**: Conducts Retrieval-Augmented Generation (RAG) searches within specific website content.

- 📚 **ArxivSearchTool**: Leverages Langchain's wrapper for ArxivAPI to locate and retrieve research papers from arxiv.org.

- 🤖 **Custom PerplexitySearchTool**: This tool utilizes Perplexity AI's API to perform AI-driven searches, returning relevant and contextually generated responses. 

#### ⛏️ Retrieval Tools

- 📄 **Summary Index**: Extracts summaries from documents, storing both the summary and all associated document nodes.

-  🔗 **Semantic Index**: Uses the Vector Store Index to create vector embeddings of document nodes, enabling semantic LLM queries.

📋 These are our core tools so far. Feel free to suggest any additional tools you think would enhance this toolkit!

In [ ]:
# Importing the necessary search tools from CrewAI
from crewai_tools import (
    SerperDevTool,         # Tool for web searches using Google results via the Serper API
    WebsiteSearchTool      # Tool for RAG (Retrieval-Augmented Generation) searches within specific website content
)

# Import BaseTool for creating custom tools and the tool decorator for additional flexibility
from crewai_tools import BaseTool, tool

# Import the arXiv API wrapper from Langchain's community utilities for academic research searches
from langchain_community.utilities import ArxivAPIWrapper

In [ ]:
# Define a langchain tool for searching research papers on arXiv
class ArxivSearchTool(BaseTool):
    name: str = "Arxiv Research Search Tool"  
    description: str = "Searches arXiv for research papers." 
    def _run(self, query: str) -> str:
        """
        Executes an arXiv search for research papers related to the given query.

        Args:
            query (str): The search query string for arXiv.
            
        Returns:
            str: A formatted list of top search results related to the query.
        """
        arxiv_search = ArxivAPIWrapper(top_k_results=10)  # Limit search results to top 10 entries
        search_result = arxiv_search.run(query)  # Run the search
        return search_result

In [ ]:
# Define a custom search tool using Perplexity AI's API for AI-driven search responses
import requests
from typing import ClassVar

class PerplexitySearchTool(BaseTool):
    name: str = "Perplexity AI Search Tool"
    description: str = (
        "Executes searches using Perplexity AI for AI-driven information retrieval. "
        "Queries should be passed as 'content' in natural language."
    )

    def _run(self, content: str) -> dict:
        """
        Executes a search query via Perplexity AI and returns the AI-generated response.

        Args:
            content (str): The search query or question in natural language.

        Returns:
            dict: JSON response from Perplexity AI containing the search results.
        """
        api_key: ClassVar[str] = os.environ["PERPLEXITY_API_KEY"]  # Retrieve Perplexity API key from environment
        url = "https://api.perplexity.ai/chat/completions"  # API endpoint for Perplexity search

        # Define the payload with search parameters
        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",  # Specify the model used for response generation
            "messages": [
                {
                    "role": "user",
                    "content": content  # Pass the search content from user input
                }
            ],
            "temperature": 0.5  # Control response variability
        }
        headers = {
            "Authorization": f"Bearer {api_key}",  # Authorization header with API key
            "Content-Type": "application/json"     # Set content type for JSON
        }

        # Make a POST request to the API and return the JSON response
        response = requests.post(url, json=payload, headers=headers)
        return response.json()

For more information on the API's usage and parameters, refer to the official [Perplexity API documentation](https://perplexity.mintlify.app/api-reference/chat-completions).

In [ ]:
# Search tool set
from crewai_tools import (
    SerperDevTool,
    WebsiteSearchTool
)

# Creating custom tools by either using the BaseTool class or the @tools decorators as follows:
from crewai_tools import BaseTool, tool

In [ ]:
# Create custom langchain tool
from langchain_community.utilities import ArxivAPIWrapper

class ArxivSearchTool(BaseTool):
    name: str = "Arxiv Research Search Tool"
    description: str = "Searches arXiv for research papers."

    def _run(self, query: str) -> str:
        """
            Runs an arXiv search with the given query
        """
        arxiv_search = ArxivAPIWrapper( 
            top_k_results = 10
        )
        search_result = arxiv_search.run(query)
        return search_result

In [ ]:
# Create custom perplexity ai search tool
import requests
from typing import ClassVar

class PerplexitySearchTool(BaseTool):
    name: str = "Perplexity AI Search Tool"
    description: str = "Pass the queries as the content and execute searches using Perplexity AI for AI-driven information retrieval."

    def _run(self, content: str) -> dict:
        """
            Runs a Perplexity AI search
        """
        api_key: ClassVar[str] = os.environ["PERPLEXITY_API_KEY"]
        url = "https://api.perplexity.ai/chat/completions"
        
        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [
                {
                    "role": "user",
                    "content": content
                }
            ],
            "temperature": 0.5
        }
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        response = requests.request("POST", url, json=payload, headers=headers)
        return response.json()   

***

***

In [ ]:
# Importing essential modules and classes for creating a RAG (Retrieval-Augmented Generation) pipeline using LlamaIndex

from llama_index.core import (
    SimpleDirectoryReader,       # Utility to read documents from a directory, transforming files into searchable content
    DocumentSummaryIndex,        # Index type that stores document summaries
    VectorStoreIndex,            # Index to store vector embeddings of documents for similarity-based search
    StorageContext,              # Utility container to manage and handle the storing of nodes, indices, and vectors
    Settings,                    # General configuration settings for LlamaIndex operations
    get_response_synthesizer     # Function to synthesize responses based on retrieved data
)

# Importing additional node parsers for flexible data processing
from llama_index.core.node_parser import *

# Importing ChromaDB, a vector database for managing and querying embeddings
import chromadb
from chromadb.config import Settings as ChromaSettings  # Chroma settings to configure database connection and storage
from llama_index.vector_stores.chroma import ChromaVectorStore  # Vector store that uses ChromaDB for efficient retrieval

# Importing OpenAI tools for embedding and language model interactions
from llama_index.llms.openai import OpenAI    # Interface for using OpenAI models for text generation
from llama_index.embeddings.openai import OpenAIEmbedding    # Tool for generating embeddings from OpenAI models

# Importing CrewAI's tool wrapper for LlamaIndex integrations
from crewai_tools import LlamaIndexTool    # Tool for using LlamaIndex functionalities in CrewAI

# Standard Python libraries for system interaction and logging
import sys
import logging

# Configuring logging to track warnings and higher-level messages
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)  # Set default logging level to WARNING
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))  # Add stream handler to ensure output to stdout

In [ ]:
# Vector store local persistent directory
persist_vector_store_path = 'chromadb'

class QueryEngineToolset:
    def __init__(self, document_dir, model_name, api_key, collection_name, load_collection_status):
        self.document_dir = document_dir
        self.model_name = model_name
        self.api_key = api_key
        self.collection_name = collection_name
        self.load_collection_status = load_collection_status
        
        Settings.llm = OpenAI(model=self.model_name, api_key=self.api_key, temperature=0)
        Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small', api_key=self.api_key)
        Settings.text_splitter = SemanticSplitterNodeParser(
            buffer_size=1,
            breakpoint_percentile_threshold=95, 
            embed_model=Settings.embed_model
        )

        # Load documents
        self.documents = self.load_documents(document_dir=self.document_dir)
         # Initialize vector store client
        self.chroma_client = self.initialize_vector_store_client()
        self.summary_index = self.create_summary_index(
            collection_name=self.collection_name, 
            load_collection_status=self.load_collection_status
        )
        self.semantic_search_index = self.create_vector_store_index(
            collection_name=self.collection_name, 
            load_collection_status=self.load_collection_status
        )
    
    def load_documents(self, document_dir):
        """
            Load documents from the specified directory.
        """
        try:
            reader = SimpleDirectoryReader(input_dir=document_dir, required_exts=['.pdf', '.docx', '.txt', '.md', 'mp3', '.mp4'])
            documents = reader.load_data()
            return documents
        except Exception as e:
            logging.error(f"Error loading documents: {e}")
            return []
        
    def initialize_vector_store_client(self):
        """
        Initialize the Chroma vector store client
        """
        try:
            chroma_client = chromadb.PersistentClient(
                path=persist_vector_store_path, 
                settings=ChromaSettings(allow_reset=True)
            )
            return chroma_client
        except Exception as e:
            logging.error(f"Error iinitializing vector store client: {e}")
            return None
        
    def initialize_chromadb_collection(self, chroma_client, collection_name, load_collection_status):
        """
        Create or load a Chroma DB collection for storing vectors.
        """
        try:
            collections_list = [col.name for col in chroma_client.list_collections()]
            if load_collection_status:
                try:
                    # Load existing collection
                    if collection_name in collections_list:
                        chromadb_collection = chroma_client.get_collection(name=collection_name)
                        logging.info(f"Loaded existing Chroma DB collection: {collection_name}")
                    else:
                        logging.error(f"Collection {collection_name} does not exist.")
                        return None
                except Exception as e:
                    logging.error(f"Error loading existing Chroma DB collection: {e}")
                    return None

            else:
                # Delete existing collection
                if collection_name in collections_list:
                    try:
                        chroma_client.delete_collection(name=collection_name)
                        logging.info(f"Deleted existing Chroma DB collection: {collection_name}")
                    except Exception as e:
                        logging.error(f"Error deleting existing Chroma DB collection: {e}")
                        return None
                
                # Create new collection
                try:
                    chromadb_collection = chroma_client.create_collection(name=collection_name)
                    logging.info(f"Created new Chroma DB collection: {collection_name}")
                except Exception as e:
                    logging.error(f"Error creating Chroma DB collection: {e}")
                    return None
            
            vector_store_instance = ChromaVectorStore(chroma_collection=chromadb_collection)
            logging.info(f"Chroma DB collection created or loaded successfully: {collection_name}")
            return vector_store_instance

        except Exception as e:
            logging.error(f"Error creating Chroma DB collection: {e}")
            return None

    def create_summary_index(self, collection_name, load_collection_status):
        """
            Create a summary index for the loaded documents.
        """
        try:
            logging.info(f"Creating or loading summary vector store with collection name: {collection_name}")
            summary_vector_store_instance = self.initialize_chromadb_collection(
                chroma_client=self.chroma_client,
                collection_name=f"{collection_name}-summary",
                load_collection_status=self.load_collection_status
            )
            summary_storage_context = StorageContext.from_defaults(vector_store=summary_vector_store_instance)
            response_synthesizer = get_response_synthesizer(
                llm=Settings.llm, response_mode="tree_summarize", use_async=True
            )

            if load_collection_status==True:
                logging.info("Loading Document summary index from existing vector store")
                summary_index =  VectorStoreIndex.from_vector_store(
                    vector_store=summary_vector_store_instance,
                    storage_context=summary_storage_context
                )
            else:
                logging.info("Creating DocumentSummaryIndex from documents")
                summary_index = DocumentSummaryIndex.from_documents(
                    documents=self.documents,
                    llm=Settings.llm,
                    transformations=[Settings.text_splitter],
                    response_synthesizer=response_synthesizer,
                    storage_context=summary_storage_context,
                    embed_model=Settings.embed_model,
                    show_progress=True,
                )
            logging.info("DocumentSummaryIndex created successfully")
            return summary_index

        except Exception as e:
            logging.error(f"Error creating summary index: {e}")
            return None

    def create_vector_store_index(self, collection_name, load_collection_status):
        """
        Create a vector store index for semantic search.
        """
        try:
            logging.info(f"Creating or loading semantic search vector store with collection name: {collection_name}")
            semantic_search_vector_store_instance = self.initialize_chromadb_collection(
                self.chroma_client,
                collection_name=f"{collection_name}-semantic_search",
                load_collection_status=self.load_collection_status
            )

            semantic_search_storage_context = StorageContext.from_defaults(
                vector_store=semantic_search_vector_store_instance
                )

            if load_collection_status==True:
                logging.info("Loading vector store index from existing vector store")
                vector_store_index = VectorStoreIndex.from_vector_store(
                    vector_store=semantic_search_vector_store_instance,
                    storage_context=semantic_search_storage_context
                )
            else:
                logging.info("Creating vector store index from documents")
                vector_store_index = VectorStoreIndex.from_documents(
                    documents=self.documents,
                    llm=Settings.llm,
                    transformations=[Settings.text_splitter],
                    embed_model=Settings.embed_model,
                    storage_context=semantic_search_storage_context,
                    show_progress=True,
                )
            logging.info("Vector store index created successfully")
            return vector_store_index
        except Exception as e:
            logging.error(f"Error creating vector store index: {e}")
            return None

    def create_tools(self):
        """
        Create query engine tools for interacting with the summary and vector store indices.
        """
        try:
            self.summary_query_engine = self.summary_index.as_query_engine(response_mode="tree_summarize", use_async=True, streaming=True)
            self.vector_store_query_engine = self.semantic_search_index.as_query_engine(similarity_top_k=5, llm=Settings.llm, streaming=True)
            self.summary_tool = LlamaIndexTool.from_query_engine(
                self.summary_query_engine,
                name="Summary Index Query Tool",
                description="Use this tool to query summaries over the given document(s)."
            )
            self.vector_store_tool = LlamaIndexTool.from_query_engine(
                self.vector_store_query_engine,
                name="Vector Store Index Query Tool",
                description="Use this tool to semantic search over the given document(s)."
            )
            return self.summary_tool, self.vector_store_tool
        except Exception as e:
            logging.error(f"Error creating query engine tools: {e}")
            return None, None

In [ ]:
# Create the GDPR RAG pipelines
gdpr_query_engine_toolset = QueryEngineToolset(
    document_dir="inputs/compliance/",
    model_name="gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"],
    collection_name="gdpr",
    load_collection_status=False
)
gdpr_summary_tool, gdpr_semantic_search_tool = gdpr_query_engine_toolset.create_tools()

## Agents, Tasks and Crew Creation

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
# Agent creation
requirements_elicitation_agent = Agent(
    config=agents_config['requirements_elicitation_agent']
)

market_researcher_agent = Agent(
    config=agents_config['market_researcher_agent'],
    tools=[
        SerperDevTool(), 
        WebsiteSearchTool(), 
        ArxivSearchTool(), 
        PerplexitySearchTool()
    ]
)

requirement_developer_agent = Agent(
    config=agents_config['requirement_developer_agent'],
)

compliance_specialist_agent = Agent(
    config=agents_config['compliance_specialist_agent'],
    tools=[
        gdpr_summary_tool, 
        gdpr_semantic_search_tool
    ]
)

quality_control_analyst_agent = Agent(
    config=agents_config['quality_control_analyst_agent'],
)

In [ ]:
# Task creation
requirement_elicitation_gathering_task = Task(
    config=tasks_config['requirement_elicitation_gathering_task'],
    agent=requirements_elicitation_agent
)

market_research_task = Task(
    config=tasks_config['market_research_task'],
    agent=market_researcher_agent,
)

requirements_development_task = Task(
    config=tasks_config['requirements_development_task'],
    agent=requirement_developer_agent
)

compliance_assessment_task = Task(
    config=tasks_config['compliance_assessment_task'],
    agent=compliance_specialist_agent
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance_task'],
    agent=quality_control_analyst_agent,
    output_file='outputs/final_BRD_v1.md'
    
)

In [ ]:
# Creating Crew
requirement_engineering_crew = Crew(
    agents=[
        requirements_elicitation_agent,
        market_researcher_agent,
        requirement_developer_agent,
        compliance_specialist_agent,
        quality_control_analyst_agent
    ],
    tasks=[
        requirement_elicitation_gathering_task,
        market_research_task,
        requirements_development_task,
        compliance_assessment_task,
        quality_assurance_task
    ],
    verbose=True
)

In [ ]:
inputs = {
    "inputs": """
    Meeting Notes: First Requirements Elicitation for Solar Analytics Mobile Application
    Date: September 22, 2024
    Attendees:
    - Product Owner (PO): Elizabeth Ogunyemi
    - Developer (Dev): Aboze Brain
    Meeting Start Time: 10:00 AM

    Summary:
    Elizabeth (Product Owner) introduced the vision for a mobile app that enables users to track, monitor, and analyze solar energy usage, aiming to provide data-driven recommendations for optimizing energy efficiency in Africa.

    Key Discussion Points:
    - Data Collection: The app will integrate with existing solar panel APIs for seamless data collection, with potential for future hardware integration.
    - Real-Time Visualization: Users should monitor energy generation and consumption in real-time, with a responsive UI for both Android and iOS platforms.
    - Platform Choice: Cross-platform options (e.g., Flutter or React Native) versus native (Swift for iOS, Kotlin for Android) were discussed, balancing speed, cost, and performance.
    - Personalized Recommendations: Machine learning could provide personalized energy-saving tips based on usage data, starting with basic recommendations and evolving over time.
    - User Experience: Consideration of data sync intervals for real-time insights without draining battery life. Notifications for significant energy events to improve user engagement.
    - Next Steps: PO and Dev agreed to begin drafting technical requirements, prioritize usability testing, and gather early user feedback.

    """
}

In [ ]:
result = requirement_engineering_crew.kickoff(inputs=inputs)

## Single Agent, Multiple Tasks Use Case

In this setup, we adopt a **single-agent, multi-task** framework, where one agent is equipped to manage several tasks. This structure capitalizes on the agent's flexibility and adaptability, enabling them to handle multiple responsibilities efficiently within a defined scope.

🔋 **Enhanced Workflow**: With this setup, the workflow remains dynamic and streamlined, as one agent can oversee multiple related tasks without needing frequent reconfiguration running tasks with dependency (`context`) and parallel (`async_execution`). This centralization keeps tasks organized and cohesive, ensuring consistent quality across all deliverables.

### Additional Implementations:
- **LLM Integration**: Modify LLMs to power the agent, using CrewAI’s LiteLLM to access a range of LLMs (see [LiteLLM documentation](https://docs.litellm.ai/docs/providers)). For this setup, we’ll use Groq and Anthropic.
- **Code writing agents**: Enable code execution for the agent, but it requires Docker to be install on safe mode.
- **Planning Models**: Incorporate models to facilitate task planning.
- **Memory**: Add memory functionality to enhance task continuity and context retention.

In [ ]:
# Load Groq and Anthropic LLMs
groq_llm = "groq/llama-3.1-70b-versatile"
anthropic_llm = "claude-3-5-sonnet-20240620"

# Add DOCXSearchTool which is RAG tool designed for semantic searching within DOCX documents.
from crewai_tools import DOCXSearchTool

doc_search_tool = DOCXSearchTool(docx='inputs/OXUM.docx')

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents_v2.yaml',
    'tasks': 'config/tasks_v2.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config_v2 = configs['agents']
tasks_config_v2 = configs['tasks']

In [ ]:
from crewai_tools import FileReadTool
from crewai_tools import CodeInterpreterTool
csv_tool = FileReadTool(file_path='inputs/solar-energy-consumption-africa.csv')

# Agent creation
requirements_elicitation_agent = Agent(
    config=agents_config_v2['requirements_elicitation_agent'],
    # llm=groq_llm,
)

data_analysis_agent = Agent(
    config=agents_config_v2['data_analysis_agent'],
    allow_code_execution=True,
    tools=[csv_tool],
    llm="gpt-4o"
)

market_researcher_agent = Agent(
    config=agents_config_v2['market_researcher_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool(), ArxivSearchTool(), PerplexitySearchTool(), doc_search_tool],
     # llm=anthropic_llm,
)

requirement_developer_agent = Agent(
    config=agents_config_v2['requirement_developer_agent'],
)

compliance_specialist_agent = Agent(
    config=agents_config_v2['compliance_specialist_agent'],
    tools=[gdpr_summary_tool, gdpr_semantic_search_tool]
)

quality_control_analyst_agent = Agent(
    config=agents_config_v2['quality_control_analyst_agent'],
)

> 🚨 **Note: Docker Desktop Settings (for Windows or Mac)**  
> **Problem**: Docker Desktop on Windows or Mac can restrict access to host directories. The `/host_mnt` structure is specific to Docker Desktop, so Docker may not be configured to access this path by default.
>  
> ✅ **Solution**: Go to Docker Desktop settings and add `/Users/dir` (or the equivalent path) as a shared folder under **Resources > File Sharing**. Restart Docker Desktop after making this change.

In [ ]:
# Task creation
brand_voice_review_task = Task(
    config=tasks_config_v2['brand_voice_review_task'],
    agent=requirements_elicitation_agent,
    # async_execution=True # To run in parallel
)

requirement_elicitation_gathering_task = Task(
    config=tasks_config_v2['requirement_elicitation_gathering_task'],
    agent=requirements_elicitation_agent,
    context=[brand_voice_review_task]
)

data_analysis_task = Task(
    config=tasks_config_v2['data_analysis_task'],
    agent=data_analysis_agent,
    context=[requirement_elicitation_gathering_task],
)

market_research_task = Task(
    config=tasks_config_v2['market_research_task'],
    context=[requirement_elicitation_gathering_task, data_analysis_task],
    agent=market_researcher_agent,  
)

requirements_development_task = Task(
    config=tasks_config_v2['requirements_development_task'],
    context=[requirement_elicitation_gathering_task, data_analysis_task, market_research_task],
    agent=requirement_developer_agent
)

compliance_assessment_task = Task(
    config=tasks_config_v2['compliance_assessment_task'],
    context=[requirements_development_task], 
    agent=compliance_specialist_agent
)

quality_assurance_task = Task(
    config=tasks_config_v2['quality_assurance_task'],
    agent=quality_control_analyst_agent,
    context=[requirements_development_task, compliance_assessment_task],
    output_file='outputs/final_BRD_v2.md'
)

In [ ]:
# Creating Crew
requirement_engineering_crew = Crew(
    agents=[
        requirements_elicitation_agent,
        data_analysis_agent,
        market_researcher_agent,
        requirement_developer_agent,
        compliance_specialist_agent,
        quality_control_analyst_agent
    ],
    tasks=[
        brand_voice_review_task,
        requirement_elicitation_gathering_task,
        data_analysis_task,
        market_research_task,
        requirements_development_task,
        compliance_assessment_task,
        quality_assurance_task
    ],
    verbose=True,
    memory=True,
    # planning=True,
    # planning_llm='gpt-4o',
    # # For hierarchical process
    # Process=Process.hierarchical,
    # manager_llm='gpt-4o',
)

In [ ]:
result = requirement_engineering_crew.kickoff(inputs=inputs)

## CrewAI Testing Feature  🚀

Testing is vital to ensure your CrewAI crew is delivering as expected. CrewAI provides built-in testing functionality to evaluate crew performance effortlessly. CrewAI makes testing straightforward with a command-line option and an in-code function:

1. **CLI Command**: Run `ccrewai test --n_iterations 5 --model gpt-4o` to initiate testing.
2. **In-Code Method**: Use `crew.test()` directly within your code to test your crew's performance.

Run these tests to get detailed performance metrics, ensuring your crew is optimized for excellence! ✨


## CrewAI Training Feature  🚀
Train your CrewAI agents by providing feedback and achieving consistent results. The training feature in CrewAI allows you to enhance your AI agents by integrating feedback and refining their performance.

1. **CLI Command**: Run `crea train -n <n_iterations> <filename> (optional)` to initiate testing.
2. **In-Code Method**: Use `crewai.train()` directly within your code to test your crew's performance.

In [ ]:
requirement_engineering_crew.test(n_iterations=2, openai_model_name='gpt-4o')                                                                                                                                 

In [ ]:
requirement_engineering_crew.train(n_iterations=1, filename='outputs/training.pkl')

🎯 Pro Tip: Regular testing and training ensure your crew is always ready to deliver exceptional performance.

### Comparing new test results

In [ ]:
requirement_engineering_crew.test(n_iterations=2, openai_model_name='gpt-4o')